In [4]:
%pip install langchain
%pip install python-dotenv
%pip install openai
%pip install tiktoken
%pip install pypdf
%pip install chromadb
%pip install fastapi nest-asyncio pyngrok uvicorn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Libraries
import os
import openai
import uvicorn
import nest_asyncio

from dotenv import load_dotenv, find_dotenv
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok

data_path = "./data/"
data = os.listdir(data_path)

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
loader = PyPDFDirectoryLoader(data_path)
docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 0
)

In [5]:
texts = text_splitter.split_documents(docs)

In [6]:
vectorstore = Chroma.from_documents(
  documents=texts,
  embedding=OpenAIEmbeddings(),
  persist_directory='./cache/'
)
vectorstore.persist()

In [13]:
print(texts[30])

page_content='Mental Disorders in Children and Adolescents 229\ntwo cognitive behavioral interventions (described be-\nlow) have met criteria for "probably efficacious":\nSelf-Control Therapy for young children and Coping\nwith Depression for adolescents.\nYoung Child Studies. Young child studies in-\nclude children in grades three to eight. Criteria for\ninclusion were based on depressive symptoms rather\nthan a diagnosis of depression. These interventions\ntypically occur in school settings over less than 20\nsessions. The clinical approaches were cognitive be-\nhavioral or psychoeducational in nature. A variety\nof interventions, typically with a cognitive behavioral\norientation, usually provided in a group format, pro-\nduced gains which ranged from four weeks to 2 years.\nThe failure of interventions reviewed by Kaslow\nand Thompson (1998) to meet criteria for "well-es-\ntablished" was based on an inability to meet the crite-\nrion of comparison with an already established treat-

In [7]:
chat_history = []
chat = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vectorstore.as_retriever()
)

In [8]:
def chatQuestion(question, chat_history):
    response = chat({"question": question,
                           "chat_history": chat_history})
    return response["answer"]

In [10]:
chatQuestion("What are the symptoms of depression?", chat_history)

' The symptoms of depression include depressed mood or loss of interest or pleasure, markedly diminished interest or pleasure in all or almost all activities, significant weight loss or gain, insomnia or hypersomnia, psychomotor agitation or retardation, fatigue or loss of energy, feelings of worthlessness or excessive or inappropriate guilt, and diminished ability to think or concentrate or indecisiveness.'